In [24]:
import pandas as pd 
import numpy as np
import copy,math

In [25]:
df = pd.read_csv('house-prices-advanced-regression-techniques/X_train.csv')

In [26]:
df

,Id,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontagenan,MasVnrAreanan,GarageYrBltnan
0,1,12.247694,0.235294,0.75,0.418208,0.366344,1.0,1.0,0.000000,0.333333,...,1.00,1.0,0.00000,0.090909,0.50,0.666667,0.75,0.0,0.0,0.0
1,2,12.109011,0.000000,0.75,0.495064,0.391317,1.0,1.0,0.000000,0.333333,...,1.00,1.0,0.00000,0.363636,0.25,0.666667,0.75,0.0,0.0,0.0
2,3,12.317167,0.235294,0.75,0.434909,0.422359,1.0,1.0,0.333333,0.333333,...,1.00,1.0,0.00000,0.727273,0.50,0.666667,0.75,0.0,0.0,0.0
3,4,11.849398,0.294118,0.75,0.388581,0.390295,1.0,1.0,0.333333,0.333333,...,1.00,1.0,0.00000,0.090909,0.00,0.666667,0.00,0.0,0.0,0.0
4,5,12.429216,0.235294,0.75,0.513123,0.468761,1.0,1.0,0.333333,0.333333,...,1.00,1.0,0.00000,1.000000,0.50,0.666667,0.75,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,12.072541,0.235294,0.75,0.400718,0.353592,1.0,1.0,0.000000,0.333333,...,1.00,1.0,0.00000,0.636364,0.25,0.666667,0.75,0.0,0.0,0.0
1456,1457,12.254863,0.000000,0.75,0.517503,0.453273,1.0,1.0,0.000000,0.333333,...,0.50,1.0,0.00000,0.090909,1.00,0.666667,0.75,0.0,0.0,0.0
1457,1458,12.493130,0.294118,0.75,0.423859,0.379597,1.0,1.0,0.000000,0.333333,...,0.75,0.5,0.16129,0.363636,1.00,0.666667,0.75,0.0,0.0,0.0
1458,1459,11.864462,0.000000,0.75,0.434909,0.393688,1.0,1.0,0.000000,0.333333,...,1.00,1.0,0.00000,0.272727,1.00,0.666667,0.75,0.0,0.0,0.0


In [27]:
y = df['SalePrice']
X = df.drop(columns='SalePrice')

In [28]:
b = 1
w_init = np.zeros(X.shape[1])
type(w_init[0])

numpy.float64

In [29]:
def predict(x,w,b):
    p = np.dot(x,w) + b
    return p

x_vec = X.iloc[0]
print(f"x_vec shape {x_vec.shape}, x_vec value: {x_vec}")

# make a prediction
f_wb = predict(x_vec,w_init, b)
print(f"f_wb shape {f_wb.shape}, prediction: {f_wb}")

x_vec shape (83,), x_vec value: Id                1.000000
MSSubClass        0.235294
MSZoning          0.750000
LotFrontage       0.418208
LotArea           0.366344
                    ...   
SaleType          0.666667
SaleCondition     0.750000
LotFrontagenan    0.000000
MasVnrAreanan     0.000000
GarageYrBltnan    0.000000
Name: 0, Length: 83, dtype: float64
f_wb shape (), prediction: 1.0


In [30]:
def compute_cost(X,y,w,b, lambda_ = 1):

    m = X.shape[0]
    n = len(w)
    cost = 0.0
    for i in range(m):
        f_wb_i = np.dot(X.iloc[i],w) + b
        cost = cost + (f_wb_i - y[i])**2
    cost = cost / (2*m)

    reg_cost = 0
    for j in range(n):
        reg_cost += (w[j]**2)                                          #scalar
    reg_cost = (lambda_/(2*m)) * reg_cost

    total_cost = cost + reg_cost

    return total_cost

In [31]:
cost = compute_cost(X, y, w_init, b, 0.7)
print(f'Cost at optimal w : {cost}')

Cost at optimal w : 60.84457538820363


In [32]:
def compute_gradient(X,y,w,b,lambda_):

    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X.iloc[i], w) + b) - y.iloc[i]   
        for j in range(n):                         
            dj_dw[j] = dj_dw[j] + err * X.iloc[i, j]    
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
        
    for j in range(n):
        dj_dw[j] = dj_dw[j] + (lambda_/m) * w[j]

    
    return dj_db, dj_dw


In [33]:
#Compute and display gradient 
tmp_dj_db, tmp_dj_dw = compute_gradient(X, y, w_init, b, 0.7)
print(f'dj_db at initial w,b: {tmp_dj_db}')
print(f'dj_dw at initial w,b: \n {tmp_dj_dw}')

dj_db at initial w,b: -11.024050901109385
dj_dw at initial w,b: 
 [-8.05004952e+03 -2.38534378e+00 -7.51348058e+00 -4.72586884e+00
 -4.20345503e+00 -1.09802121e+01 -1.05041178e+01 -1.60260156e+00
 -3.86764531e+00 -1.10166321e+01 -1.42153435e+00 -3.45681805e-01
 -5.46943431e+00 -4.35436273e+00 -1.09126155e+01 -8.05847582e+00
 -7.40648296e+00 -6.29624950e+00 -6.30280391e+00 -2.91040266e+00
 -4.25155702e+00 -2.31537305e+00 -2.01067304e-01 -7.36556887e+00
 -6.85985995e+00 -5.43949002e+00 -7.29723561e-01 -5.18132813e+00
 -1.04379820e+01 -7.66963332e+00 -6.98040392e+00 -8.16710148e+00
 -4.53075072e+00 -7.83478050e+00 -8.78532795e-01 -8.48219543e+00
 -3.48353597e-01 -2.69368224e+00 -1.92542619e+00 -1.08576599e+01
 -8.71359041e+00 -1.03413615e+01 -1.06293810e+01 -5.02028619e+00
 -1.87937118e+00 -1.11352205e-01 -5.71491754e+00 -1.57930931e+00
 -3.16884865e-01 -5.79473096e+00 -2.14194073e+00 -3.95847944e+00
 -3.84150746e+00 -5.61380084e+00 -4.17927437e+00 -1.05743475e+01
 -2.29461586e+00 -5.0957

In [34]:
def gradient_descent(X,y,w_in, b_in, lambda_, cost_function, gradient_function, alpha, num_iters):

    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db,dj_dw = gradient_function(X, y, w, b, lambda_)   ##None

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               ##None
        b = b - alpha * dj_db               ##None
      
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( cost_function(X, y, w, b, lambda_))

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}   ")
        
    return w, b, J_history #return final w,b and J history for graphing

In [35]:
# initialize parameters
initial_w = np.zeros_like(w_init)
print(w_init)
print(initial_w)

initial_b = 0.
# some gradient descent settings
iterations = 400
alpha = 5.0e-7
lambda_ = 0.7
# run gradient descent 
w_final, b_final, J_hist = gradient_descent(X, y, initial_w, initial_b, lambda_,
                                                    compute_cost, compute_gradient, 
                                                    alpha, iterations)
print(f"b,w found by gradient descent: {b_final:0.2f},{w_final} ")
m,_ = X.shape

for i in range(m):
    print(f"prediction: {np.dot(X.iloc[i], w_final) + b_final:0.2f}, target value: {y.iloc[i]}")

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Iteration    0: Cost    40.67   
Iteration   40: Cost    18.16   
Iteration   80: Cost    18.16   
Iteration  120: Cost    18.16   
Iteration  160: Cost    18.15   
Iteration  200: Cost    18.15   
Iteration  240: Cost    18.14   
Iteration  280: Cost    18.14   
Iteration  320: Cost    18.13   
Iteration  360: Cost    18.13   
b,w found by gradient descent: 0.00,[ 1.23288364e-02  1.28815157e-04  4.16778909e-04  2.67449755e-04
  2.38070915e-04  6.10647831e-04  5.86623680e-04  1.00877351e

In [36]:
y_pred = np.dot(X,w_final) + b_final

In [37]:
from sklearn.metrics import mean_squared_error, r2_score

In [38]:
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 36.245259141732134
R-squared: -226.31068965534433
